# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sys
sys.path.append('C:/Users/User/Desktop/AG News')
from data_preprocessing import *

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

In [ ]:
textcuboid=np.load('./1-Channel textcuboid_ag(elmo).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_ag(elmo).npy')

# 전처리

In [3]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [4]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [8]:
tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [ ]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

# 모델 빌드

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [5]:
input_layer = Input(shape=(46, 256))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 46, 256)]         0         
                                                                 
 conv1d (Conv1D)             (None, 46, 1024)          263168    
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                             

In [15]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=50,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/50
59/59 [==============================] - 1s 7ms/step - loss: 0.5619 - accuracy: 0.8073 - val_loss: 0.3524 - val_accuracy: 0.8770 - lr: 0.0010
Epoch 2/50
59/59 [==============================] - 0s 5ms/step - loss: 0.3085 - accuracy: 0.8996 - val_loss: 0.3425 - val_accuracy: 0.8830 - lr: 0.0010
Epoch 3/50
59/59 [==============================] - 0s 5ms/step - loss: 0.2633 - accuracy: 0.9125 - val_loss: 0.3099 - val_accuracy: 0.8930 - lr: 0.0010
Epoch 4/50
59/59 [==============================] - 0s 5ms/step - loss: 0.2259 - accuracy: 0.9236 - val_loss: 0.3178 - val_accuracy: 0.8900 - lr: 0.0010
Epoch 5/50
59/59 [==============================] - 0s 5ms/step - loss: 0.1914 - accuracy: 0.9335 - val_loss: 0.2915 - val_accuracy: 0.8890 - lr: 0.0010
Epoch 6/50
59/59 [==============================] - 0s 5ms/step - loss: 0.1672 - accuracy: 0.9423 - val_loss: 0.3377 - val_accuracy: 0.8920 - lr: 0.0010
Epoch 7/50
59/59 [==============================] - 0s 5ms/step - loss: 0.1386 - a

In [16]:
y_test=np.array(y_test)

In [17]:
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 1ms/step - loss: 0.3785 - accuracy: 0.9205


[0.37847012281417847, 0.9204999804496765]

In [18]:
from keras.models import load_model
model = load_model('best_model_5.h5')
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 1ms/step - loss: 0.2311 - accuracy: 0.9183


[0.23106372356414795, 0.9182500243186951]